In [1]:

bonus_root = '/network/group/aopp/predict/TIP016_PAXTON_RPSPEEDY/ML4L/ECMWF_files/raw/BonusClimate/'
#Wetlands
wetlands = ['COPERNICUS', 'CAMA','ORCHIDEE','monthlyWetlandAndSeasonalWater_minusRiceAllCorrected_waterConsistent']
lakes = ['CL_ECMWFAndJRChistory','yearlyCL']


In [2]:
import glob
import pandas as pd
import xarray as xr
import sys

import re

def natural_sort(l): 
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)', key)]
    return sorted(l, key=alphanum_key)





def get_ERA_hour(ERA_month,ERA_constant,t):
    
    """
    Extract an hour of ERA data
    """
    
    #Filter month of ERA data to an hour
    time_filter = (ERA_month.time == t)
    ERA_hour = ERA_month.where(time_filter,drop=True)

    #Join on the constant data, first setting the time coordinate
    ERA_constant = ERA_constant.assign_coords({"time": (((ERA_hour.time)))}) #Update the time for the merge.
    ERA_hour = xr.merge([ERA_hour,ERA_constant]).load() #Explicitly load 

    #Now filter to get land values only 
    land_filter = (ERA_hour.lsm > 0.5)
    ERA_hour = ERA_hour.where(land_filter,drop=True)

    #And covert longitude to long1
    ERA_hour = ERA_hour.assign_coords({"longitude": (((ERA_hour.longitude + 180) % 360) - 180)})

    return ERA_hour

root = '/network/group/aopp/predict/TIP016_PAXTON_RPSPEEDY/ML4L/ECMWF_files/raw/'

#Time constant ERA data. This is different for v15 and v20 data.
versions = ["v15", "v20"]
ERA_constant_dict = {}
for v in versions:
    f = root +f'processed_data/ERA_timeconstant/ERA_constants_{v}.nc'
    ds = xr.open_dataset(f) #NetCDF file of features which are constant for each gridpoint
    
    ERA_constant_dict[v] = ds
    ds.close()



In [3]:
#Time variable ERA data
ERA_folder = root+'processed_data/ERA_timevariable/'
ERA_files = natural_sort(glob.glob(ERA_folder+'*'))
f_ERA = ERA_files[0]
print(f_ERA)
ERA_month = xr.open_dataset(f_ERA,engine='cfgrib',backend_kwargs={'indexpath': ''})

    
#Get all times in that month of data, hourly grain
timestamps = pd.to_datetime(ERA_month.time) 

/network/group/aopp/predict/TIP016_PAXTON_RPSPEEDY/ML4L/ECMWF_files/raw/processed_data/ERA_timevariable/ERA_0.grib


In [9]:
import numpy as np
ERA_month_timestamp = np.unique(timestamps.month)[0] #There should only be one value, an interger in range 1-12

In [18]:
#Wetlands
wetlands_ds
#Filter year of wetlands data to a month
wetlands_time_filter = (wetlands_ds.time == ERA_month_timestamp)
wetlands_month = wetlands_ds.where(wetlands_time_filter,drop=True)

In [19]:
wetlands_month

<xarray.Dataset>
Dimensions:                                                               (time: 1, values: 542080)
Coordinates:
  * time                                                                  (time) int64 ...
    step                                                                  float64 ...
    surface                                                               float64 ...
    latitude                                                              (values) float64 ...
    longitude                                                             (values) float64 ...
    valid_time                                                            (time) float64 ...
Dimensions without coordinates: values
Data variables:
    COPERNICUS                                                            (time, values) float32 ...
    CAMA                                                                  (time, values) float32 ...
    ORCHIDEE                                                              (time, values) float32 ...
    monthlyWetlandAndSeasonalWater_minusRiceAllCorrected_waterConsistent  (time, values) float32 ...

In [6]:
def select_correct_MODIS_file(t):
    
    """We have to be careful with the dateline. This function
       figures out which MODIS file to load."""

    
    #Get the hour
    utc_hour = t.hour
    
    
    #Due to crossing of the datetime, some times will be saved different date
    if utc_hour < min_hours[satellite]:
        file_date = t  - np.timedelta64(1,'D')
    elif utc_hour > max_hours[satellite]:
        file_date = t  + np.timedelta64(1,'D')
    else:
        file_date = t
        
    #Create a string which will be used to open file
    y = pd.to_datetime(file_date).year
    m = pd.to_datetime(file_date).month
    d = pd.to_datetime(file_date).day
    date_string = f'{y}-{m:02}-{d:02}'
    
    return date_string



#These dictionaries describe the local hour of the satellite
local_times = {"aquaDay":"13:30",
               "terraDay":"10:30",
               "terraNight":"22:30",
               "aquaNight":"01:30"
              }
# and are used to load the correct file for dealing with the date-line.
min_hours = {"aquaDay":2,
            "terraDay":-1,
            "aquaNight":-1,
            "terraNight":11}
max_hours = {"aquaDay":24,
            "terraDay":22,
            "aquaNight":13,
            "terraNight":24}


In [ ]:
import numpy as np
for w in wetlands:
    fname = bonus_root+w+'/wetlandf'
    print(fname)
    ds_wetland= xr.open_dataset(fname,engine='cfgrib',decode_times=False,backend_kwargs={'indexpath': ''}) 
    
    
    
    #Rename the parameter
    ds_wetland = ds_wetland.cldiff.rename(f'cldiff_{w}') #This is now a dataarray
    display(ds_wetland.time)
    #Fix the time 
    ds_wetland['time'] = np.arange(1,12+1) #i.e. what month it it?
    
    #Select only the correct time 
    chosen_month = 
    
    display(ds_wetland)
    

/network/group/aopp/predict/TIP016_PAXTON_RPSPEEDY/ML4L/ECMWF_files/raw/processed_data/ERA_timevariable/ERA_0.grib


In [39]:
timestamps = pd.to_datetime(ERA_month.time) 
chosen_month = np.unique(timestamps.month)

In [34]:
np.arange(1,12+1) #1-12 inclusive

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

In [15]:
import numpy as np


for v in versions:
    f = root +f'processed_data/ERA_timeconstant/ERA_constants_{v}.nc'
    ds = xr.open_dataset(f) #NetCDF file of features which are constant for each gridpoint
    
    ERA_constant_dict[v] = ds




wetlands_dict = {}
wetlands_ds = xr.Dataset()
for w in wetlands:
    fname = bonus_root+w+'/wetlandf'
    print(fname)
    ds_wetland= xr.open_dataset(fname,engine='cfgrib',decode_times=False,backend_kwargs={'indexpath': ''}) 
    
    
    
    #Rename the parameter
    ds_wetland = ds_wetland.cldiff.rename(f'cldiff_{w}') #This is now a dataarray

    #Fix the time 
    ds_wetland['time'] = np.arange(1,12+1) #i.e. what month it it?
    
    wetlands_ds[w] = ds_wetland
    #wetlands_dict[w] = ds_wetland

/network/group/aopp/predict/TIP016_PAXTON_RPSPEEDY/ML4L/ECMWF_files/raw/BonusClimate/COPERNICUS/wetlandf
/network/group/aopp/predict/TIP016_PAXTON_RPSPEEDY/ML4L/ECMWF_files/raw/BonusClimate/CAMA/wetlandf
/network/group/aopp/predict/TIP016_PAXTON_RPSPEEDY/ML4L/ECMWF_files/raw/BonusClimate/ORCHIDEE/wetlandf
/network/group/aopp/predict/TIP016_PAXTON_RPSPEEDY/ML4L/ECMWF_files/raw/BonusClimate/monthlyWetlandAndSeasonalWater_minusRiceAllCorrected_waterConsistent/wetlandf


In [16]:
wetlands_ds

<xarray.Dataset>
Dimensions:                                                               (time: 12, values: 542080)
Coordinates:
  * time                                                                  (time) int64 ...
    step                                                                  float64 ...
    surface                                                               float64 ...
    latitude                                                              (values) float64 ...
    longitude                                                             (values) float64 ...
    valid_time                                                            (time) float64 ...
Dimensions without coordinates: values
Data variables:
    COPERNICUS                                                            (time, values) float32 ...
    CAMA                                                                  (time, values) float32 ...
    ORCHIDEE                                                              (time, values) float32 ...
    monthlyWetlandAndSeasonalWater_minusRiceAllCorrected_waterConsistent  (time, values) float32 ...

In [14]:
wetlands_dict['COPERNICUS']

<xarray.DataArray 'cldiff_COPERNICUS' (time: 12, values: 542080)>
[6504960 values with dtype=float32]
Coordinates:
  * time        (time) int64 1 2 3 4 5 6 7 8 9 10 11 12
    step        float64 ...
    surface     float64 ...
    latitude    (values) float64 ...
    longitude   (values) float64 ...
    valid_time  (time) float64 ...
Dimensions without coordinates: values
Attributes:
    GRIB_paramId:                    200026
    GRIB_dataType:                   an
    GRIB_numberOfPoints:             542080
    GRIB_typeOfLevel:                surface
    GRIB_stepUnits:                  1
    GRIB_stepType:                   instant
    GRIB_gridType:                   reduced_gg
    GRIB_N:                          320
    GRIB_NV:                         0
    GRIB_cfName:                     unknown
    GRIB_cfVarName:                  cldiff
    GRIB_gridDefinitionDescription:  Gaussian latitude/longitude
    GRIB_missingValue:               9999
    GRIB_name:                       Lake cover difference
    GRIB_pl:                         [  18   25   36   40   45   50   60   64...
    GRIB_shortName:                  cldiff
    GRIB_units:                      (0 - 1)
    long_name:                       Lake cover difference
    units:                           (0 - 1)
    standard_name:                   unknown

In [5]:




#Empty dict. We will append the resulting dfs here
dfs = {"v15":[],
       "v20":[]}
for t in timestamps:

    print(t)
    #Get an hour of MODIS data
    date_string = select_correct_MODIS_file(t) #For this datetime, which MODIS file should be opened? 
    if date_string == '2017-12-31': continue #skip we dont have this day

    
    #ERA constant
    v = "v15"
    ERA_constant = ERA_constant_dict[v]
    

    
    
    #Get an hour of ERA data
    ERA_hour = get_ERA_hour(ERA_month,ERA_constant,t)
    
    
    #ERA_df = ERA_hour.to_dataframe().reset_index().dropna()





grib_ls: Command not found.
